In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import os
import sys
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
# import matplotlib
# matplotlib.use('Agg')
import matplotlib.pyplot as plt
import argparse
from datasets import *
from nets import *
from configs import *
from random import shuffle
from torch.autograd import Variable
from tensorboardX import SummaryWriter
import matplotlib.pyplot as plt

from datasets import *

In [3]:
from nets import *

In [4]:
class args:
    batch=32
    history=32
    numsegments=10
    hidden=256
    lr=0.0005
    epochs=120

In [5]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [6]:
model = Series(
    batchsize=args.batch,
    historylen=args.history,
    numsegments=args.numsegments + 1,
    hiddensize=args.hidden).to(device).double()

In [7]:
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=args.lr)

In [ ]:
train_refs = create_dataset_knodes(max_nodes=2)
for eii in range(args.epochs):
    shuffle(train_refs)
    losses = []
    nBatches = len(train_refs)//args.batch
    for bii in range(0, len(train_refs) - args.batch, args.batch):
        model.train()
        
        batch_refs = train_refs[bii:bii+args.batch]
        batch_seq, batch_lbls = knodes_batch(batch_refs)
        
        batch_seq = np.transpose(batch_seq, [2, 0, 1])
        batch_seq = Variable(torch.from_numpy(batch_seq), requires_grad=True) \
            .to(device).double()
        batch_lbls = torch.from_numpy(batch_lbls).unsqueeze(2).to(device)

        lstm_state = model.init_lstms(device=device)

        preds, _ = model(batch_seq, lstm_state)

        loss = criterion(preds, batch_lbls)
        losses.append(loss.item())
        n_iter = eii*(nBatches)+bii
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        sys.stdout.write('[E:%d/%d] B:%d/%d  loss: %.2f          \r' % (
            eii+1, args.epochs,
            bii//args.batch+1, nBatches,
            loss.item() * 100.0**2,
        ))
        sys.stdout.flush()